In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import re
import string

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
dataset_path = '/content/all_ai_tool.csv'
data = pd.read_csv(dataset_path)

In [ ]:
'''data['Useable For'] = data['Useable For'].fillna('')
data['Major Category'] = data['Major Category'].fillna('')

data['combined_features'] = data['Useable For'] + ' ' + data['Major Category']
'''

"data['Useable For'] = data['Useable For'].fillna('')\ndata['Major Category'] = data['Major Category'].fillna('')\n\ndata['combined_features'] = data['Useable For'] + ' ' + data['Major Category']\n"

In [ ]:
def clean_text(text):
  if isinstance(text, str):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower().strip()
    return text
  else:
    return ''


data['Useable For'] = data['Useable For'].apply(clean_text)
data['Major Category'] = data['Major Category'].apply(clean_text)
data['combined_features'] = data['Useable For'] + ' ' + data['Major Category']

NameError: name 'data' is not defined

In [ ]:
data

,AI Tool Name,Description,Free/Paid/Other,Useable For,Charges,Review,Tool Link,Major Category,combined_features
0,Farm3D,Learning Articulated 3D Animals by Distilling ...,Free,experiments d,NaN,NaN,https://farm3d.github.io/?utm_source=futureped...,other,experiments d other
1,Mailbutler Ai,"AI-powered Smart Assistant for Outlook, Gmail ...",Freemium,email assistant,$ 4.95/mo,1,https://www.mailbutler.io/smart-assistant/?utm...,text,email assistant text
2,Maverick,Send customers personalized videos at scale.,Paid,ecommerce audio editing,$ 100/mo,NaN,https://www.trymaverick.com/?utm_source=future...,audio,ecommerce audio editing audio
3,0Xai,Stable Diffusion Prompt And Images Generator,Free,image generator prompts,NaN,NaN,https://0xai.quest?utm_source=futurepedia&utm_...,image,image generator prompts image
4,10Web,AI-powered WordPress platform for automated we...,Freemium,lowcodenocode,NaN,NaN,https://10web.io/?utm_source=futurepedia&utm_m...,code,lowcodenocode code
...,...,...,...,...,...,...,...,...,...
4964,Zoom Iq,Capture meaningful and actionable insights fro...,Contact for Pricing,summarizer email assistant,NaN,2,https://blog.zoom.us/zoom-iq-smart-companion/?...,text,summarizer email assistant text
4965,Zoomscape Ai,ZoomScape AI is an AI-powered tool that enable...,Free Trial,presentations,$5,NaN,https://zoomscape.ai/?utm_source=futurepedia&u...,business,presentations business
4966,Zust Qr,Enables users to craft artistic QR codes by in...,Free,image generator,NaN,NaN,https://qr.zust.ai/?utm_source=futurepedia&utm...,image,image generator image
4967,Zuva,Contract analysis and insight generation.,Free Trial,legal assistant,$ 50/mo,1,https://zuva.ai/widget/try-it/?utm_source=futu...,business,legal assistant business


In [ ]:
def preprocess_query(query):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(query.lower())
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

In [ ]:
user_query = "I need a tool that can generate images."
processed_query = preprocess_query(user_query)
print(f"Processed Query: {processed_query}")

Processed Query: need tool generate images


In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['combined_features'])

In [ ]:
def expand_query(query):
  words = query.split()
  expanded_words = []
  for word in words:
    synonyms = wordnet.synsets(word)
    expanded_words.extend(set([lemma.name() for syn in synonyms for lemma in syn.lemmas()]))
  expanded_query = ' '.join(set(words + expanded_words))
  return expanded_query

In [ ]:
def recommend_ai_tools(user_query, top_n=5):
  user_query_cleaned = clean_text(user_query)
  expanded_query = expand_query(user_query_cleaned)
  query_vector = tfidf_vectorizer.transform([expanded_query])
  cosine_sim = cosine_similarity(query_vector, tfidf_matrix)
  top_indices = cosine_sim[0].argsort()[-top_n:][::-1]
  return data.iloc[top_indices]

In [ ]:
'''query_tfidf = tfidf_vectorizer.transform([processed_query])

similarity_scores = cosine_similarity(query_tfidf, combined_features_tfidf)
'''

'query_tfidf = tfidf_vectorizer.transform([processed_query])\n\nsimilarity_scores = cosine_similarity(query_tfidf, combined_features_tfidf)\n'

In [ ]:
'''top_indices = np.argsort(similarity_scores[0])[::-1][:5]
top_tools = data.iloc[top_indices]
'''

'top_indices = np.argsort(similarity_scores[0])[::-1][:5]\ntop_tools = data.iloc[top_indices]\n'

In [ ]:
recommendations = recommend_ai_tools(processed_query)

In [ ]:
print("Top recommended tools:")
print(top_tools[['AI Tool Name', 'Description', 'Free/Paid/Other', 'Useable For', 'Major Category']])

Top recommended tools:


NameError: name 'top_tools' is not defined

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import re
import string
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the dataset
dataset_path = '/content/all_ai_tool.csv'  # Replace with your actual file path
data = pd.read_csv(dataset_path)

# Fill missing values in 'Useable For' and 'Major Category' with an empty string
data['Useable For'] = data['Useable For'].fillna('')
data['Major Category'] = data['Major Category'].fillna('')

# Combine 'Useable For' and 'Major Category' into a single column for processing
data['combined_features'] = data['Useable For'] + ' ' + data['Major Category']

# Function to clean text
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = text.lower().strip()
        return text
    else:
        return ''

data['combined_features'] = data['combined_features'].apply(clean_text)


def preprocess_query(query):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(query.lower())
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)


def expand_query(query):
    words = query.split()
    expanded_words = []
    for word in words:
        synonyms = wordnet.synsets(word)
        expanded_words.extend(set([lemma.name() for syn in synonyms for lemma in syn.lemmas()]))
    expanded_query = ' '.join(set(words + expanded_words))
    return expanded_query

# Vectorize the text using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
combined_features_tfidf = tfidf_vectorizer.fit_transform(data['combined_features'])


def recommend_ai_tools(user_query, top_n=5):
    user_query_cleaned = clean_text(user_query)
    expanded_query = expand_query(user_query_cleaned)
    print("Hello")
    print(expanded_query)
    query_vector = tfidf_vectorizer.transform([expanded_query])
    cosine_sim = cosine_similarity(query_vector, combined_features_tfidf)
    top_indices = cosine_sim[0].argsort()[-top_n:][::-1]
    return data.iloc[top_indices]

# Example usage
user_query = "I need music generator and text generator"
processed_query = preprocess_query(user_query)
recommendations = recommend_ai_tools(processed_query)

# Display the recommended tools
print("Top recommended tools:")
print(recommendations[['AI Tool Name', 'Free/Paid/Other', 'Useable For', 'Major Category']])

Hello
necessitate medicine require demand music want pauperization involve ask author text indigence call_for textbook need take euphony text_edition motive postulate pauperism generator penury source school_text schoolbook motivation textual_matter
Top recommended tools:
                 AI Tool Name        Free/Paid/Other  \
3283             Pollinations                   Free   
3714             Rotor Videos                   Paid   
3345                   Prodia               Freemium   
2774  Moji Writing Assistant   Freemium | Free-Trial   
4893                     Wzrd                   Paid   

                                            Useable For Major Category  
3283    / image generator    / music    / video gene...          audio  
3714                       / music    / video generator          audio  
3345              / image generator    / music    / art          audio  
2774    / general writing    / image generator    / ...          audio  
4893                     

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import re
import string
import nltk
import time

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the dataset
dataset_path = '/content/all_ai_tool.csv'  # Replace with your actual file path
data = pd.read_csv(dataset_path)

# Fill missing values in 'Useable For' and 'Major Category' with an empty string
data['Useable For'] = data['Useable For'].fillna('')
data['Major Category'] = data['Major Category'].fillna('')

# Combine 'Useable For' and 'Major Category' into a single column for processing
data['combined_features'] = data['Useable For'] + ' ' + data['Major Category']

# Function to clean text
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = text.lower().strip()
        return text
    else:
        return ''

data['combined_features'] = data['combined_features'].apply(clean_text)

def preprocess_query(query):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(query.lower())
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

def expand_query(query):
    words = query.split()
    expanded_words = []
    for word in words:
        synonyms = wordnet.synsets(word)
        expanded_words.extend(set([lemma.name() for syn in synonyms for lemma in syn.lemmas()]))
    expanded_query = ' '.join(set(words + expanded_words))
    return expanded_query

# Vectorize the text using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
combined_features_tfidf = tfidf_vectorizer.fit_transform(data['combined_features'])

def recommend_ai_tools(user_query, top_n=5):
    user_query_cleaned = clean_text(user_query)
    expanded_query = expand_query(user_query_cleaned)
    query_vector = tfidf_vectorizer.transform([expanded_query])
    cosine_sim = cosine_similarity(query_vector, combined_features_tfidf)
    top_indices = cosine_sim[0].argsort()[-top_n:][::-1]
    return data.iloc[top_indices]

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Example usage
user_query = "i need help in video"
processed_query = preprocess_query(user_query)
start_time = time.time()
recommendations = recommend_ai_tools(processed_query)
end_time = time.time()
response_time = end_time - start_time
print(f"Response Time: {response_time:.4f} seconds")


# Display the recommended tools
print("Top recommended tools:")
print(recommendations[['AI Tool Name', 'Free/Paid/Other', 'Useable For', 'Major Category']])

# Evaluation function
def evaluate_model(test_data, tfidf_vectorizer, combined_features_tfidf):
    y_true = []
    y_pred = []
    for index, row in test_data.iterrows():
        query = row['combined_features']
        true_category = row['Major Category']
        recommended_tools = recommend_ai_tools(query)
        predicted_category = recommended_tools['Major Category'].iloc[0] if not recommended_tools.empty else None
        y_true.append(true_category)
        y_pred.append(predicted_category)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    return accuracy, precision, recall

# Calculate evaluation metrics
accuracy, precision, recall = evaluate_model(test_data, tfidf_vectorizer, combined_features_tfidf)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Response Time: 0.0029 seconds
Top recommended tools:
       AI Tool Name      Free/Paid/Other  \
3447        Qlip.Ai           Free Trial   
4642       Vimeo Ai                 Paid   
1641    Flawless Ai  Contact for Pricing   
2807          Movmi             Freemium   
4028  Spikes Studio             Freemium   

                                 Useable For Major Category  
3447    / video editing    / video generator          video  
4642    / video editing    / video generator          video  
1641    / video editing    / video generator          video  
2807    / video editing    / video generator          video  
4028    / video editing    / video generator          video  
Accuracy: 0.9798792756539235
Precision: 0.9771018432045416
Recall: 0.9798792756539235
